<a href="https://colab.research.google.com/github/MukterHo/Data-Clean-Data-Analysis-with-data-Model/blob/main/My_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', '{:.2f}'.format)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shree1992/housedata")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/housedata


In [ ]:
df = pd.read_csv(path + "/data.csv")

In [ ]:
df.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city',
       'statezip', 'country'],
      dtype='object')

In [ ]:
df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
count,4600.00,4600.00,4600.00,4600.00,4600.00,4600.00,4600.00,4600.00,4600.00,4600.00,4600.00,4600.00,4600.00
mean,551962.99,3.40,2.16,2139.35,14852.52,1.51,0.01,0.24,3.45,1827.27,312.08,1970.79,808.61
std,563834.70,0.91,0.78,963.21,35884.44,0.54,0.08,0.78,0.68,862.17,464.14,29.73,979.41
min,0.00,0.00,0.00,370.00,638.00,1.00,0.00,0.00,1.00,370.00,0.00,1900.00,0.00
25%,322875.00,3.00,1.75,1460.00,5000.75,1.00,0.00,0.00,3.00,1190.00,0.00,1951.00,0.00
50%,460943.46,3.00,2.25,1980.00,7683.00,1.50,0.00,0.00,3.00,1590.00,0.00,1976.00,0.00
75%,654962.50,4.00,2.50,2620.00,11001.25,2.00,0.00,0.00,4.00,2300.00,610.00,1997.00,1999.00
max,26590000.00,9.00,8.00,13540.00,1074218.00,3.50,1.00,4.00,5.00,9410.00,4820.00,2014.00,2014.00


In [ ]:
def outliers(df,col):
  q1,q3 = np.percentile(df[col],[25,75])
  iqr = q3-q1
  lower_bound = q1 - (1.5 * iqr)
  upper_bound = q3 + (1.5 * iqr)
  df=df[(df[col]<=upper_bound) & (df[col]>=lower_bound)]
  return df

df = outliers(df, 'price')
df = outliers(df, 'sqft_lot')

In [ ]:
df.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city',
       'statezip', 'country'],
      dtype='object')

In [ ]:
df['saling_date']=pd.to_datetime(df['date']).dt.year

In [ ]:
df.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city',
       'statezip', 'country', 'saling_date'],
      dtype='object')

In [ ]:
df['House_age']=df['saling_date']-df['yr_built']


In [ ]:
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country,saling_date,House_age
0,2014-05-02 00:00:00,313000.00,3.00,1.50,1340,7912,1.50,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA,2014,59
2,2014-05-02 00:00:00,342000.00,3.00,2.00,1930,11947,1.00,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA,2014,48
3,2014-05-02 00:00:00,420000.00,3.00,2.25,2000,8030,1.00,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA,2014,51
4,2014-05-02 00:00:00,550000.00,4.00,2.50,1940,10500,1.00,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA,2014,38
5,2014-05-02 00:00:00,490000.00,2.00,1.00,880,6380,1.00,0,0,3,880,0,1938,1994,522 NE 88th St,Seattle,WA 98115,USA,2014,76


In [ ]:
#df['is_renovated'] = np.where(df['yr_renovated'] > 0, 1, 0)
df["is_reno"]=np.where(df['yr_renovated']>0,1,0)

In [ ]:
colo_drop= ['date', 'yr_renovated', 'yr_built', 'street', 'country']
df.drop(colo_drop,axis=1,inplace=True)

In [ ]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,city,statezip,saling_date,House_age,is_reno
0,313000.00,3.00,1.50,1340,7912,1.50,0,0,3,1340,0,Shoreline,WA 98133,2014,59,1
2,342000.00,3.00,2.00,1930,11947,1.00,0,0,4,1930,0,Kent,WA 98042,2014,48,0
3,420000.00,3.00,2.25,2000,8030,1.00,0,0,4,1000,1000,Bellevue,WA 98008,2014,51,0
4,550000.00,4.00,2.50,1940,10500,1.00,0,0,4,1140,800,Redmond,WA 98052,2014,38,1
5,490000.00,2.00,1.00,880,6380,1.00,0,0,3,880,0,Seattle,WA 98115,2014,76,1


In [ ]:
X=df.drop('price',axis=1)
y=df['price']


In [ ]:
X_train,y_train,X_test,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
numerical_ferecher=x.select_dtypes(include=['int64','float64']).columns
categorical_ferecher=x.select_dtypes(include=['object']).columns


In [ ]:
preprocessor=ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),numerical_ferecher),
        ('cat',OneHotEncoder(handle_unknown='ignore'),categorical_ferecher)
    ]
)

In [ ]:
models={
    'LinearRegression':LinearRegression(),
    'DecisionTreeRegressor':DecisionTreeRegressor(),
    'RandomForestRegressor':RandomForestRegressor(),
    'GradientBoostingRegressor':GradientBoostingRegressor(),
    'XGBRegressor':XGBRegressor()
}

In [ ]:
trained_models={}

In [ ]:
# prompt: problem solve

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # Keep other columns (like the newly created features)
)

pipelines = {name: Pipeline(steps=[('preprocessor', preprocessor), ('regressor', model)])
             for name, model in models.items()}

for name, pipeline in pipelines.items():
    print(f"Training {name}...")
    pipeline.fit(X_train, y_train)
    trained_models[name] = pipeline
    y_pred = pipeline.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"{name} Metrics:")
    print(f"  MAE: {mae:.2f}")
    print(f"  MSE: {mse:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  R2 Score: {r2:.2f}")
    print("-" * 30)


Training LinearRegression...
LinearRegression Metrics:
  MAE: 77512.35
  MSE: 16404059257.53
  RMSE: 128078.33
  R2 Score: 0.65
------------------------------
Training DecisionTreeRegressor...
DecisionTreeRegressor Metrics:
  MAE: 112422.87
  MSE: 33637997648.60
  RMSE: 183406.65
  R2 Score: 0.28
------------------------------
Training RandomForestRegressor...
RandomForestRegressor Metrics:
  MAE: 80098.89
  MSE: 15158540082.97
  RMSE: 123120.02
  R2 Score: 0.67
------------------------------
Training GradientBoostingRegressor...
GradientBoostingRegressor Metrics:
  MAE: 88846.65
  MSE: 16939804575.46
  RMSE: 130153.00
  R2 Score: 0.64
------------------------------
Training XGBRegressor...
XGBRegressor Metrics:
  MAE: 77679.13
  MSE: 14941707264.00
  RMSE: 122236.28
  R2 Score: 0.68
------------------------------


In [ ]:
# prompt: Predicted Result use model fit

# Use the trained XGBoost model to predict on the test set
xgb_pipeline = trained_models['XGBRegressor']
y_pred_xgb = xgb_pipeline.predict(X_test)

print("\nXGBoost Model Predicted Prices:")
print(y_pred_xgb[:15]) # Print the first 10 predicted prices


XGBoost Model Predicted Prices:
[309496.6  544433.25 393180.44 183031.72 294893.16 427372.2  251587.44
 494844.53 327502.28 710565.7  406858.53 429218.34 755389.8  440885.97
 608185.1 ]


In [ ]:
# prompt: Predictvalue

# Select a trained model (e.g., the Random Forest Regressor)
best_model_name = 'XGBRegressor'
best_model = trained_models[best_model_name]

# Example data for prediction (replace with your actual data)
# This should be a pandas DataFrame with the same columns as X_train before preprocessing
new_data = pd.DataFrame({
    'bedrooms': [3],
    'bathrooms': [2.5],
    'sqft_living': [1800],
    'sqft_lot': [5000],
    'floors': [2],
    'waterfront': [0],
    'view': [0],
    'condition': [3],
    'grade': [7],
    'sqft_above': [1800],
    'sqft_basement': [0],
    'zipcode': ['98001'],
    'lat': [47.1234],
    'long': [-122.5678],
    'sqft_living15': [1800],
    'sqft_lot15': [5000],
    'saling_date': [2015],  # Example year
    'House_age': [10],      # Example age
    'is_reno': [0],
    'city': ['Seattle'] , # Example city
    'statezip': ['WA 98059'] , # Example state
})

# Ensure the columns are in the same order as the training data (except 'price')
# This is important for the preprocessor
new_data = new_data[X_train.columns]

# Predict the price
predicted_price = best_model.predict(new_data)

print(f"Predicted price using {best_model_name}: ${predicted_price[0]:,.2f}")

Predicted price using XGBRegressor: $539,283.31


In [ ]:
# Select a trained model (e.g., the Random Forest Regressor)
best_model_name = 'RandomForestRegressor'
best_model = trained_models[best_model_name]

# Example data for prediction (replace with your actual data)
# This should be a pandas DataFrame with the same columns as X_train before preprocessing
new_data = pd.DataFrame({
    'bedrooms': [3],
    'bathrooms': [2.5],
    'sqft_living': [1800],
    'sqft_lot': [5000],
    'floors': [2],
    'waterfront': [0],
    'view': [0],
    'condition': [3],
    'grade': [7],
    'sqft_above': [1800],
    'sqft_basement': [0],
    'zipcode': ['98001'],
    'lat': [47.1234],
    'long': [-122.5678],
    'sqft_living15': [1800],
    'sqft_lot15': [5000],
    'saling_date': [2015],  # Example year
    'House_age': [10],      # Example age
    'is_reno': [0],
    'city': ['Seattle'] , # Example city
    'statezip': ['WA 98059'] , # Example state
})

# Ensure the columns are in the same order as the training data (except 'price')
# This is important for the preprocessor
new_data = new_data[X_train.columns]

# Predict the price
predicted_price = best_model.predict(new_data)

print(f"Predicted price using {best_model_name}: ${predicted_price[0]:,.2f}")

Predicted price using RandomForestRegressor: $504,791.46


In [ ]:
X_train.columns
X_train.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,city,statezip,saling_date,House_age,is_reno
2528,4.00,2.75,2280,15347,1.00,0,0,5,2280,0,Newcastle,WA 98059,2014,54,0
2348,4.00,2.50,2300,19250,1.00,0,0,4,2300,0,North Bend,WA 98045,2014,59,1
1852,3.00,1.00,920,14400,1.00,0,0,4,920,0,Auburn,WA 98001,2014,37,0
3093,2.00,1.00,870,11100,1.00,0,0,3,870,0,Seattle,WA 98146,2014,74,1
868,3.00,1.50,1710,4500,1.50,0,0,3,1410,300,Seattle,WA 98118,2014,86,1
